# Focos de Calor - Incêndios Florestais de 1998 - 2017

Base extraída do site de dados públicos do governo https://dados.gov.br/dataset/sistema-nacional-de-informacoes-florestais-snif

In [113]:
import numpy as np
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go

#Função para mostrar o gráfico no notebook
py.init_notebook_mode(connected=True)

In [114]:
archive = 'rf_incendiosflorestais_focoscalor_estados_1998-2017.csv'

base = pd.read_csv(archive)
base['date'] = pd.to_datetime(base.date)
base

,ano,state,month,number,date
0,1998,Acre,Janeiro,0,1998-01-01
1,1999,Acre,Janeiro,0,1999-01-01
2,2000,Acre,Janeiro,0,2000-01-01
3,2001,Acre,Janeiro,0,2001-01-01
4,2002,Acre,Janeiro,0,2002-01-01
...,...,...,...,...,...
6449,2012,Tocantins,Dezembro,128,2012-01-01
6450,2013,Tocantins,Dezembro,85,2013-01-01
6451,2014,Tocantins,Dezembro,223,2014-01-01
6452,2015,Tocantins,Dezembro,373,2015-01-01


In [115]:
# Informações sobre oa dados
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ano     6454 non-null   int64         
 1   state   6454 non-null   object        
 2   month   6454 non-null   object        
 3   number  6454 non-null   int64         
 4   date    6454 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 252.2+ KB


In [116]:
base.describe()

,ano,number
count,6454.000000,6454.000000
mean,2007.461729,567.559653
std,5.746654,1648.372747
min,1998.000000,0.000000
25%,2002.000000,9.000000
50%,2007.000000,55.000000
75%,2012.000000,284.000000
max,2017.000000,25963.000000


In [119]:
# Função para Série Temporal

def plot_forest_fires_timeline(df):
    
    dataTemp = df.groupby(['date']).agg({'number': 'sum'}).reset_index()
    
    dataTemp['3-years-SMA'] = dataTemp['number'].rolling(window=3).mean()
    
    graph = go.Scatter(x = dataTemp['date'],
                    y = dataTemp['number'],
                    mode = 'markers+lines',
                    name = 'Quantidade de Incêndios Florestais'
                    )
    
    graph_sma = go.Scatter(x = dataTemp['date'],
                    y = dataTemp['3-years-SMA'],
                    mode = 'markers+lines',
                    name = 'Média Móvel do Triênio'
                    )
    
    data = [graph, graph_sma]

    layout = go.Layout(title = 'Série Temporal dos Focos de Calor',
                      xaxis = {'title': 'Ano'},
                      yaxis = {'title': 'Número de Focos de Calor'},
                      height=600, 
                      width=1000 )
            
    fig = go.Figure(data=data, layout=layout)
    
    return py.iplot(fig)
    

In [120]:
plot_forest_fires_timeline(base)

In [121]:
# Função para Gráfico de Barras

def plot_forest_fires_state(df):
    
    dataTemp = df.groupby(['state']).agg({'number': 'sum'}).sort_values(
        by=['number'], ascending=False).reset_index()
    
    
    graph = go.Bar(x = dataTemp['state'],
                    y = dataTemp['number'],
                    text = dataTemp['number'],
                    textposition='auto'
                    )
    
    
    data = [graph]

    layout = go.Layout(title = 'Número de Focos de Calor por Estado',
                      xaxis = {'title': 'Estado'},
                      yaxis = {'title': 'Número de Focos de Calor'},
                      height=600, 
                      width=1000 )
            
    fig = go.Figure(data=data, layout=layout)
    
    return py.iplot(fig)

In [122]:
plot_forest_fires_state(base)

In [123]:
# Função para Gráfico de Barras

def plot_forest_fires_month(df):
    
    dataTemp = df.groupby(['month']).agg({'number': 'sum'}).sort_values(
        by=['number'], ascending=False).reset_index()
    
    
    graph = go.Bar(x = dataTemp['month'],
                    y = dataTemp['number'],
                    text = dataTemp['number'],
                    textposition='auto'
                    )
    
    
    data = [graph]

    layout = go.Layout(title = 'Número de Focos de Calor por Mês',
                      xaxis = {'title': 'Estado'},
                      yaxis = {'title': 'Número de Focos de Calor'},
                      height=600, 
                      width=900)
            
    fig = go.Figure(data=data, layout=layout)
    
    return py.iplot(fig)

In [124]:
plot_forest_fires_month(base)

In [125]:
# Histograma dos focos de calor

def plot_histogram(df):

    graph = go.Histogram(x=df['number']) 

                          
    data = [graph]
    
    layout = go.Layout(title = 'Histograma dos Números de Focos de Calor',
                   xaxis = {'title': 'Distribuição'},
                   yaxis = {'title': 'Frequência'})
    
    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig)

In [126]:
plot_histogram(base)

In [127]:
# Boxplot do número de focos de calor

def plot_box_plot(df):
    
    graph = go.Box(y = df['number'],
                name = 'Focos de Calor')
    
    data = [graph]
    
    layout = go.Layout(title = 'BoxPlot dos Números de Focos de Calor',
                      height=800, 
                      width=800)
    
    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig)

In [128]:
plot_box_plot(base)